## 3.5 Subclassing UserDict (custome mappings)

![](https://velog.velcdn.com/images/jeguring/post/880edf91-2671-4f1c-a1df-f88459ab0bd8/image.png)

dict보다 UserDict를 상속해서 매핑형을 만드는 것이 훨씬 쉽습니다. 매핑에 추가한 키를 문자열로 저장하기 위해서 ex3-7.StrKeyDict0을 확장했던 것처럼 UserDict의 값을 평가할 수 있습니다.

build-in형에서는 아무런 문제없이 상속할 수 있는 메소드들을 오버라이드해야 하는 구현의 특이성 때문에 dict보다는 UserDict를 상속하는 것이 낫습니다. 
* dict 등의 내장 클래스를 상속할 때 발생하는 문제 : 12.1절

UserDict는 dict를 상속하지 않고 내부에 실제 항목을 담고 있는 data라고 하는 dict 객체를 갖고 있습니다. 이렇게 구현함으로써 __setitem__() 등의 스페셜 메소드를 구현할 때 발생하는 원치않는 재귀 호출을 피할 수 있으며, __contains__() 메소드를 간단히 구현할 수 있습니다.




`UserDict` 클래스가 `MutableMapping`을 상속하므로, `StrKeyDict`는 결국 `UserDict, MutableMapping, Mapping`을 상속하게 되어 매핑의 모든 기능을 가지게 된다.  

`Mapping`은 추상 베이스 클래스(ABC)임에도 불구하고 유용한 구상(구체적인) 메서드를 다수 제공한다.   
다음을 살펴보자

UserDict로 구현한 StrKeyDict는 위에서 구현한 StrKeyDict0보다 간단하지만 더 많은 작업을 하도록 구현할 수 있습니다.  
StrKeyDict는 모든 키를 str타입으로 저장함으로써 비문자열 키로 객체를 생성하거나 갱신할 때 발생할 수 있는 예기치 못한 문제들을 피하게 해줍니다.

- `MutableMapping.update()` :
  이 메소드는 직접 호출할 수도 있지만, 다른 매핑이나 (key, value) 쌍의 반복형 및 키워드 인수에서 객체를 로딩하기 위해 `__init__()`에 의해 사용될 수도 있습니다. 이 메소드는 항목을 추가하기 위해서 `self[key] = value` 구문을 사용하므로 결국 서브클래스에서 구현한 `__setitem__()` 메소드를 호출하게 됩니다.
- `Mapping.get()` : 
  StrKeyDict0에서는 `__getitem__()`과 일치하는 결과를 가져오기 위해 `get()` 메소드를 직접 구현해야 했지만, StrKeyDict에서는 `StrKeyDict0.get()`과 완전히 동일하게 구현된 `Mapping.get()`을 상속 받습니다.


In [ ]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()


In [2]:
import collections

class StrKeyDict(collections.UserDict): # StrKeyDict : UserDict 상속
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key): # 저장된 키가 모두 str형이므로, 위에서 self.keys()를 호출한 방법과 달리, self.data에서 바로 조회할 수 있다.
        return str(key) in self.data
    
    def __setitem__(self, key, item): # 모든 키를 str형으로 반환하므로, 연산을 self.data에 위임할 때 더 간단히 작성할 수 있다.
        self.data[str(key)] = item


In [3]:
u = StrKeyDict(zip(["1","korea",3], ["d","d","r"]))
u.data

{'1': 'd', 'korea': 'd', '3': 'r'}

In [4]:
# 숫자형을 문자로 바꿔주기 위한 것인건가 그냥?
print(1 in u)
print(korea in u)

True


NameError: name 'korea' is not defined

## 3.7 Immutable Mapping
원래 딕셔너리는 mutable임.  
Immutable Dict 생성을 해볼 것임.   
hashtable : 적은 리소스로 많은 데이터를 효율적으로 관리   
Dict -> Key 중복 허용 X 
Set -> 중복 허용 X 

In [5]:
# immutable Dict
from types import MappingProxyType # 읽기 전용의 딕셔너리
d = {'key1' : 'value1'}

# Read Only
d_frozen = MappingProxyType(d)
print(d, id(d))
print(d_frozen, id(d_frozen))

{'key1': 'value1'} 4421560896
{'key1': 'value1'} 4425659920


In [6]:
# 수정 불가 d_frozen는 immutable Dict - mappingproxy
d_frozen['key2'] = 'value2' # d_frozen을 변경할 순 없다.

TypeError: 'mappingproxy' object does not support item assignment

In [7]:
# 수정 가능 d는 Dict
d['key2'] = 'value2'
print(d) # mutable
print(d_frozen) # 동적인 d_frozen은 d에 대한 변경을 바로 반영한다.

{'key1': 'value1', 'key2': 'value2'}
{'key1': 'value1', 'key2': 'value2'}


#### 보충자료: Dictionary Views
dict의 인스턴스 메소드인 `.keys()`, `.values()`, 그리고 `.items()`는 각각 dict_keys, dict_values, dict_items라는 클래스의 인스턴스를 반환합니다. 이 딕셔너리 뷰는 dict의 구현에서 내부 데이터 구조체의 **read-only projections**입니다.

In [8]:
d = dict(zip(['a', 'b', 'c'], [10, 20, 30]))
values = d.values()
print(values, len(values))
print('==='*30)
# view객체는 dynamic proxy(동적 프록시) => 원본 dict이 업데이트되면, view에서도 변경된 걸 확인 가능.
d['z'] = 99
print(d)
print(values, len(values))

dict_values([10, 20, 30]) 3
{'a': 10, 'b': 20, 'c': 30, 'z': 99}
dict_values([10, 20, 30, 99]) 4


#### numpy에서 `copy()` vs `view()`
- `copy()` : copy본을 수정해도 원본에 영향을 미치지 않는다. 
- `view()` : view본을 수정할 경우 원본도 수정됨. base를 공유

In [1]:
import numpy as np
a = np.arange(5)
b = a.view()
b[0] = 100

print('b:', b)
print('a도 변경됨:', a)

b: [100   1   2   3   4]
a도 변경됨: [100   1   2   3   4]


In [2]:
import numpy as np
a = np.arange(5)
b = a.copy()
b[0] = 100

print('b:', b)
print('a는 변경안됨:', a)

b: [100   1   2   3   4]
a는 변경안됨: [0 1 2 3 4]


## 3.8 집합 이론
- set 
- frozenset : set의 불변형 버전
  
set의 기본적인 사용은 지나감

In [3]:
# 기본적인 set 만들기 => pass
s1 = {'Apple', 'Orange', 'Apple', 'Orange', 'Kiwi'}
s2 = set(['Apple', 'Orange', 'Apple', 'Orange', 'Kiwi'])
print(s1 == s2)
s3 = {3}
print(type(s3))

True
<class 'set'>


In [4]:
# 원소가 하나도 없을 떈 딕셔너리가 되어버리니, 공집합을 표현하고 싶을 땐 반드시 set()로 표기하기
s4 = {}
print(type(s4)) # 원소가 하나도 없을 땐 딕셔너리가 되버림

s4 = set() # 따라서 공집합일 경우 반드시 set()로 표기해야 한다.
print(type(s4))

<class 'dict'>
<class 'set'>


In [5]:
# frozenset
s5 = frozenset({'Apple', 'Orange', 'Apple', 'Orange', 'Kiwi'})

# 원래 set에는 추가됨
s1.add('Melon')
print('set기본형 - s1:', s1)

# 그러나 frozenset에는 추가 안됨
s5.add('Melon')
print(s5)

set기본형 - s1: {'Apple', 'Melon', 'Orange', 'Kiwi'}


AttributeError: 'frozenset' object has no attribute 'add'

In [6]:
print(s1, type(s1))
print(s2, type(s2))
print(s3, type(s3))
print(s4, type(s4))
print(s5, type(s5))

{'Apple', 'Melon', 'Orange', 'Kiwi'} <class 'set'>
{'Apple', 'Orange', 'Kiwi'} <class 'set'>
{3} <class 'set'>
set() <class 'set'>
frozenset({'Apple', 'Orange', 'Kiwi'}) <class 'frozenset'>


In [7]:
# 선언 최적화
# 바이트 코드 -> 파이썬 인터프리터가 실행
from dis import dis

print('------')
print(dis('{10}'))
print('------')
print(dis('set([10])'))

------
  1           0 LOAD_CONST               0 (10)
              2 BUILD_SET                1
              4 RETURN_VALUE
None
------
  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (10)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE
None


### 3.8.2 지능형 집합(Comprehending Set)

In [8]:
# 지능형 집합(Comprehending Set)
from unicodedata import name
print({chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')})
print({name(chr(i), '') for i in range(32, 256) if 'SIGN' in name(chr(i), '')})

{'+', '®', '§', '¬', '=', '<', '£', '#', '±', '°', '÷', '¥', '×', '¢', '©', '>', '%', 'µ', '¤', '¶', '$'}
{'DIVISION SIGN', 'DOLLAR SIGN', 'CURRENCY SIGN', 'COPYRIGHT SIGN', 'NOT SIGN', 'DEGREE SIGN', 'EQUALS SIGN', 'MICRO SIGN', 'PERCENT SIGN', 'YEN SIGN', 'PLUS SIGN', 'SECTION SIGN', 'MULTIPLICATION SIGN', 'CENT SIGN', 'PILCROW SIGN', 'LESS-THAN SIGN', 'GREATER-THAN SIGN', 'NUMBER SIGN', 'REGISTERED SIGN', 'PLUS-MINUS SIGN', 'POUND SIGN'}


In [10]:
import jovian
jovian.commit(filename='C3 - Presentation.ipynb', project='hongbi/fluent-python-C3-pr')

<IPython.core.display.Javascript object>

[jovian] Creating a new project "hongb1/fluent-python-C3-pr"
[jovian] Committed successfully! https://jovian.ai/hongbi/fluent-python-c3-pr


'https://jovian.ai/hongbi/fluent-python-c3-pr'